#  `infer_subc` :  a tool/framework for infering segmented organelles from multi-channel florescent 3D images.

## ❹. QUANTIFICATION 📏📐🧮

SUBCELLULAR COMPONENT METRICS
-  extent 
-  size
-  shape
-  position



This notebook illustrates various "stats" routines developed to quantify and gather statistics about the segmentations.



## IMPORTS

In [1]:
# top level imports
from pathlib import Path
import os, sys


import napari

### import local python functions in ../infer_subc
sys.path.append(os.path.abspath((os.path.join(os.getcwd(), '..'))))

from infer_subc.core.file_io import (read_czi_image,
                                        export_inferred_organelle,
                                        import_inferred_organelle,
                                        export_tiff,
                                        list_image_files)

from infer_subc.core.img import *
from infer_subc.utils.stats import *
from infer_subc.utils.stats_helpers import *

from infer_subc.organelles import * 

import time
%load_ext autoreload
%autoreload 2



In [2]:
# NOTE:  these "constants" are only accurate for the testing MCZ dataset
from infer_subc.constants import (TEST_IMG_N,
                                    NUC_CH ,
                                    LYSO_CH ,
                                    MITO_CH ,
                                    GOLGI_CH ,
                                    PEROX_CH ,
                                    ER_CH ,
                                    LD_CH ,
                                    RESIDUAL_CH )              


## SETUP

In [3]:
# this will be the example image for testing the pipeline below
test_img_n = TEST_IMG_N

# build the datapath
# all the imaging data goes here.
data_root_path = Path(os.path.expanduser("~")) / "Projects/Imaging/data"

# linearly unmixed ".czi" files are here
in_data_path = data_root_path / "raw"
im_type = ".czi"

# get the list of all files
img_file_list = list_image_files(in_data_path,im_type)
test_img_name = img_file_list[test_img_n]

# save output ".tiff" files here
out_data_path = data_root_path / "out"

if not Path.exists(out_data_path):
    Path.mkdir(out_data_path)
    print(f"making {out_data_path}")

In [4]:
img_data,meta_dict = read_czi_image(test_img_name)

# get some top-level info about the RAW data
channel_names = meta_dict['name']
img = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']

source_file = meta_dict['file_name']

/Users/ergonyc/miniforge3/envs/napari10/lib/python3.10/site-packages/ome_types/_convenience.py:112: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


Now get the single "optimal" slice of all our organelle channels....

## get the inferred cellmask, nuclei and cytoplasm objects

(takes < 1 sec)

Builde the segmentations in order




In [6]:

###################
# CELLMASK, NUCLEI, CYTOPLASM, NUCLEUS
###################
nuclei_obj =  get_nuclei(img_data,meta_dict, out_data_path)
cellmask_obj = get_cellmask(img_data, nuclei_obj, meta_dict, out_data_path)
cyto_mask = get_cytoplasm(nuclei_obj , cellmask_obj , meta_dict, out_data_path)



loaded  inferred 3D `nuc`  from /Users/ergonyc/Projects/Imaging/data/out 
loaded  inferred 3D `cell`  from /Users/ergonyc/Projects/Imaging/data/out 
`cytoplasm` object not found: /Users/ergonyc/Projects/Imaging/data/out/ZSTACK_PBTOhNGN2hiPSCs_BR1_N19_Unmixed-cytoplasm.tiff
starting segmentation...
saved file: None
inferred cytoplasm. wrote None
inferred cytoplasm in (0.05) sec


-------------------------
## regionprops

`skimage.measure.regionprops` provides the basic tools nescessary to quantify our segmentations.

First lets see what works in 3D.  

> Note: the names of the regionprops correspond to the 2D analysis even for those which are well defined in 3D.  i.e. "area" is actually "volume" in 3D, etc.

Lets see which possible measures are sensible for 3D or volumetric with regionprops

In [7]:
if False:
    labels = label(nuclei_obj )
    rp = regionprops(labels, intensity_image=img_data[NUC_CH])

    supported = [] 
    unsupported = []

    for prop in rp[0]:
        try:
            rp[0][prop]
            supported.append(prop)
        except NotImplementedError:
            unsupported.append(prop)

    print("Supported properties:")
    print("  " + "\n  ".join(supported))
    print()
    print("Unsupported properties:")
    print("  " + "\n  ".join(unsupported))

Supported properties:
-  area
-  bbox
-  bbox_area
-  centroid
-  convex_area
-  convex_image
-  coords
-  equivalent_diameter
-  euler_number
-  extent
-  feret_diameter_max
-  filled_area
-  filled_image
-  image
-  inertia_tensor
-  inertia_tensor_eigvals
-  intensity_image
-  label
-  local_centroid
-  major_axis_length
-  max_intensity
-  mean_intensity
-  min_intensity
-  minor_axis_length
-  moments
-  moments_central
-  moments_normalized
-  slice
-  solidity
-  weighted_centroid
-  weighted_local_centroid
-  weighted_moments
-  weighted_moments_central
-  weighted_moments_normalized
-
Unsupported properties:
-  eccentricity
-  moments_hu
-  orientation
-  perimeter
-  perimeter_crofton
-  weighted_moments_hu


-----------------
## basic stats

### per-organelle


- regionprops 


### summary stats

Now we want to get a list of our organelle names, segmentations, intensities (florescence)

In [12]:
# names of organelles we have
organelle_names = ["nuc","lyso", "mito","golgi","perox","ER","LD"]

get_methods  = [get_nuclei,
                get_lyso,
                get_mito,
                get_golgi,
                get_perox,
                get_ER,
                get_LD]

# load all the organelle segmentations
organelles = [meth(img_data,meta_dict, out_data_path) for meth in get_methods]

# get the intensities
organelle_channels = [NUC_CH, LYSO_CH,MITO_CH,GOLGI_CH,PEROX_CH,ER_CH,LD_CH]

intensities = [img_data[ch] for ch in organelle_channels]


loaded  inferred 3D `nuc`  from /Users/ergonyc/Projects/Imaging/data/out 
loaded  inferred 3D `lyso`  from /Users/ergonyc/Projects/Imaging/data/out 
loaded lyso in (0.00) sec
loaded  inferred 3D `mito`  from /Users/ergonyc/Projects/Imaging/data/out 
loaded  inferred 3D `golgi`  from /Users/ergonyc/Projects/Imaging/data/out 
starting segmentation...
loaded  inferred 3D `perox`  from /Users/ergonyc/Projects/Imaging/data/out 
loaded peroxisome in (0.00) sec
loaded  inferred 3D `ER`  from /Users/ergonyc/Projects/Imaging/data/out 
loaded  inferred 3D `LD`  from /Users/ergonyc/Projects/Imaging/data/out 


In [14]:
#lyso
org_img = intensities[1]        
org_obj = _assert_uint16_labels(organelles[1])

# A_stats_tab, rp = get_simple_stats_3D(A,mask)
a_stats_tab, rp = get_summary_stats_3D(org_obj, org_img, organelle_mask)




-----------------
## CONTACTS (cross-stats)

### organelle cross stats


- regionprops 



- intersect for A vs all other organelles Bi
  - regionprops on A ∩ Bi

   
- contacts?
  - dilate then intersect?
  - loop through each sub-object for each 



In [ ]:


#mito
b = _assert_uint16_labels(organelles[2])
nmi = organelle_names[2]

cross_tab = get_aXb_stats_3D(org_obj, b, organelle_mask) 
shell_cross_tab = get_aXb_stats_3D(org_obj, b, organelle_mask, use_shell_a=True)

merged_tab = pd.concat([cross_tab,shell_cross_tab])
merged_tab.insert(loc=0,column='organelle_b',value=nmi )



-----------------
## DISTRIBUTION  


### Radial distribution 

### 2D projection of inferred objects (and masks, florescence image)

Segment image in 3D;
sum projection of binary image; 
create 5 concentric rings going from the edge of the nuclie to the edge of the cellmask (ideally these will be morphed to cellmask/nuclei shape as done in CellProfiler); 
measure intensity per ring (include nuclei as the center area to measure from)/ring area; 
the normalized measurement will act as a frequency distribution of that organelle starting from the nuclei bin going out to the cell membrane - 
Measurements needed: mean, median, and standard deviation of the frequency will be calculated

- pre-processing
  1. Make 2D sum projection of binary segmentation
  2. Create 5 (default) bins linearly between edge of the nuclei to the edge of the cellmask - these are somewhat like rings morphed to the shape of the nuclei and cellmask, or more accurately like terrain lines of the normalized radial distance beween teh edge of the nuclei and the edge of the cellmask.
  3. Use nucleus + concentric rings to mask the 2D sum project into radial distribution regions: nuclei = bin 1, ... largest/outter most ring = bin 6. See similar concept in CellProfiler: https://cellprofiler-manual.s3.amazonaws.com/CellProfiler-4.2.5/modules/measurement.html?highlight=distribution#module-cellprofiler.modules.measureobjectintensitydistribution"	
   


The logic was borrowed from CellProfiler, but alorithm somewhate simplified by making assumpitions of doing all estimates over a single cellmask (single cell).   Most of the code should be capable of performing the more complicated multi-object versions as CellProfiler does.  Although this functionality is untested the source code was left in this more complex format in case it might be updated for this functionality in the future.




## Zernicky distributions...
- get the magnitude and phase for the zernike 
- he Zernike features characterize the distribution of intensity across the object. For instance, Zernike 1,1 has a high value if the intensity is low on one side of the object and high on the other. The zernike magnitudes feature records the rotationally invariant degree magnitude of the moment and the zernike phase feature gives the moment’s orientation

`zernike_degree` (default = 9) chooses how many moments to calculate.


The logic was borrowed from CellProfiler, but alorithm greatly simplified by making assumpitions of doing all estimates over a single cellmask (single cell)

In [ ]:
# csv_path = out_data_path / f"{o}_{meta_dict["file_name"].split('/')[-1].split('.')[0]}_stats.csv"
Path(meta_dict['file_name']).name

'ZSTACK_PBTOhNGN2hiPSCs_BR1_N19_Unmixed.czi'

In [ ]:
organelles[0].shape, organelle_names[0]

((15, 768, 768), 'nuclei')

In [15]:
test_org = 1

# args 
cellmask_obj
nuclei_obj
organelle_mask = cyto_mask
organelle_name = organelle_names[test_org]
org_obj = organelles[test_org]
org_img = intensities[test_org]

n_rad_bins = 5
n_zernike = 9

target = organelle_name

In [16]:
        # now get radial stats
rad_stats,z_stats, bin_index = get_radial_stats(        
                                cellmask_obj,
                                organelle_mask,
                                org_obj,
                                org_img,
                                target,
                                nuclei_obj,
                                n_rad_bins,
                                n_zernike
                                )


In [ ]:
viewer = napari.view_image(bin_idx)


### depth - summary
Segment image in 3D;
measure area fraction of each organelle per Z slice;
these measurements will act as a frequency distribution of that organelle starting from the bottom of the cellmask (not including neurites) to the top of the cellmask;
measurements: mean, median, and standard deviation of the frequency distribution	

- pre-processing
  1. subtract nuclei from the cellmask --> cellmask cytoplasm
  2. mask organelle channels with cellmask cytoplasm mask

- per-object measurements
  - For each Z slice in the masked binary image measure:
    1. organelle area
    2. cellmask cytoplasm area

- per-object calculations
  - For each Z slice in the masked binary image: organelle area / cellmask cytoplasm area

- per cell summary
  1. create a frequency table with the z slice number on the x axis and the area fraction on the y axis
  2. Measure the frequency distribution's mean, median, and standard deviation for each cell"

In [ ]:
viewer.add_image(cellmask_obj>0)

<Image layer 'Image' at 0x2bf465db0>

In [106]:

d_stats = get_depth_stats(        
                cellmask_obj,
                organelle_mask,
                org_obj,
                org_img,
                target,
                nuclei_obj
                )
  

# putting it all together

`make_organelle_stat_tables` prototype

In [13]:

from infer_subc.utils.stats import _assert_uint16_labels

organelle_to_colname = {"nuc":"NU", "lyso": "LY", "mito":"MT", "golgi":"GL", "perox":"PR", "ER":"ER", "LD":"LD", "cell":"CM", "cyto":"CY", "nucleus": "N1","nuclei":"NU",}

def _make_organelle_stat_tables(
    organelle_names: List[str],
    organelles: List[np.ndarray],
    intensities: List[np.ndarray],
    nuclei_obj:np.ndarray, 
    cellmask_obj:np.ndarray,
    organelle_mask: np.ndarray, 
    out_data_path: Path, 
    source_file: str,
    n_rad_bins: Union[int,None] = None,
    n_zernike: Union[int,None] = None,
) -> int:
    """
    get summary and all cross stats between organelles `a` and `b`
    calls `get_summary_stats_3D`
    """
    count = 0
    org_stats_tabs = []
    for j, target in enumerate(organelle_names):
        org_img = intensities[j]        
        org_obj = _assert_uint16_labels(organelles[j])

        # A_stats_tab, rp = get_simple_stats_3D(A,mask)
        a_stats_tab, rp = get_summary_stats_3D(org_obj, org_img, organelle_mask)
        a_stats_tab.insert(loc=0,column='organelle',value=target )
        a_stats_tab.insert(loc=0,column='ID',value=source_file.stem )

        # add the touches for all other organelles
        # loop over Bs
        merged_tabs = []
        for i, nmi in enumerate(organelle_names):
            if i != j:
                # get overall stats of intersection
                # print(f"  b = {nmi}")
                count += 1
                # add the list of touches
                b = _assert_uint16_labels(organelles[i])

                ov = []
                b_labs = []
                labs = []
                for idx, lab in enumerate(a_stats_tab["label"]):  # loop over A_objects
                    xyz = tuple(rp[idx].coords.T)
                    cmp_org = b[xyz]
                    
                    # total number of overlapping pixels
                    overlap = sum(cmp_org > 0)
                    # overlap?
                    labs_b = cmp_org[cmp_org > 0]
                    b_js = np.unique(labs_b).tolist()

                    # if overlap > 0:
                    labs.append(lab)
                    ov.append(overlap)
                    b_labs.append(b_js)

                cname = organelle_to_colname[nmi]
                # add organelle B columns to A_stats_tab
                a_stats_tab[f"{cname}_overlap"] = ov
                a_stats_tab[f"{cname}_labels"] = b_labs  # might want to make this easier for parsing later

                #####  2  ###########
                # get cross_stats

                cross_tab = get_aXb_stats_3D(org_obj, b, organelle_mask) 
                shell_cross_tab = get_aXb_stats_3D(org_obj, b, organelle_mask, use_shell_a=True)
                            
                # cross_tab["organelle_b"]=nmi
                # shell_cross_tab["organelle_b"]=nmi
                #  Merge cross_tabs and shell_cross_tabs 
                # merged_tab = pd.merge(cross_tab,shell_cross_tab, on="label_")
                merged_tab = pd.concat([cross_tab,shell_cross_tab])
                merged_tab.insert(loc=0,column='organelle_b',value=nmi )

                merged_tabs.append( merged_tab )


        #  Now append the 
        # csv_path = out_data_path / f"{source_file.stem}-{target}_shellX{nmi}-stats.csv"
        # e_stats_tab.to_csv(csv_path)
        # stack these tables for each organelle
        crossed_tab = pd.concat(merged_tabs)
        # csv_path = out_data_path / f"{source_file.stem}-{target}X{nmi}-stats.csv"
        # stats_tab.to_csv(csv_path)
        crossed_tab.insert(loc=0,column='organelle',value=target )
        crossed_tab.insert(loc=0,column='ID',value=source_file.stem )

        # now get radial stats
        rad_stats,z_stats, _ = get_radial_stats(        
                cellmask_obj,
                organelle_mask,
                org_obj,
                org_img,
                target,
                nuclei_obj,
                n_rad_bins,
                n_zernike
                )

        d_stats = get_depth_stats(        
                cellmask_obj,
                organelle_mask,
                org_obj,
                org_img,
                target,
                nuclei_obj
                )
      
        proj_stats = pd.merge(rad_stats, z_stats,on=["organelle","mask"])
        proj_stats = pd.merge(proj_stats, d_stats,on=["organelle","mask"])
        proj_stats.insert(loc=0,column='ID',value=source_file.stem )

        # write out files... 
        # org_stats_tabs.append(A_stats_tab)
        csv_path = out_data_path / f"{source_file.stem}-{target}-stats.csv"
        a_stats_tab.to_csv(csv_path)

        csv_path = out_data_path / f"{source_file.stem}-{target}-cross-stats.csv"
        crossed_tab.to_csv(csv_path)

        csv_path = out_data_path / f"{source_file.stem}-{target}-proj-stats.csv"
        proj_stats.to_csv(csv_path)

        count += 1

    print(f"dumped {count}x3 organelle stats ({organelle_names}) csvs")
    return count

In [107]:
n_rad_bins = 5
n_zernike = 9



from infer_subc.utils.stats_helpers import make_organelle_stat_tables

In [108]:
file_count = make_organelle_stat_tables(organelle_names, 
                                      organelles,
                                      intensities, 
                                      nuclei_obj,
                                      cellmask_obj,
                                      cyto_mask, 
                                      out_data_path, 
                                      source_file,
                                      n_rad_bins=5,
                                      n_zernike=9)



dumped 49x3 organelle stats (['nuc', 'lyso', 'mito', 'golgi', 'perox', 'ER', 'LD']) csvs


/Users/ergonyc/Projects/Imaging/infer-subc/infer_subc/utils/stats.py:668: RuntimeWarning: invalid value encountered in divide
  magnitude = np.sqrt(vr * vr + vi * vi) / pixels.sum()


In [109]:
target = organelle_names[1]

csv_path = out_data_path / f"{source_file.stem}-{target}-stats.csv"

mito_table = pd.read_csv(csv_path)
mito_table.head()

,Unnamed: 0,ID,organelle,label,max_intensity,mean_intensity,min_intensity,volume,equivalent_diameter,centroid-0,...,MT_overlap,MT_labels,GL_overlap,GL_labels,PR_overlap,PR_labels,ER_overlap,ER_labels,LD_overlap,LD_labels
0,0,ZSTACK_PBTOhNGN2hiPSCs_BR1_N19_Unmixed,lyso,2,65535,16752.953982,0,30836,38.906289,6.450642,...,2346,"[1, 2]",2019,[1],77,"[3, 5, 6, 15, 17, 20, 21]",3715,[1],0,[]
1,1,ZSTACK_PBTOhNGN2hiPSCs_BR1_N19_Unmixed,lyso,5,21639,10853.279793,2242,1544,14.340013,2.794041,...,0,[],0,[],0,[],136,[1],0,[]
2,2,ZSTACK_PBTOhNGN2hiPSCs_BR1_N19_Unmixed,lyso,6,5773,2776.759036,480,83,5.412025,1.445783,...,0,[],0,[],0,[],0,[],0,[]
3,3,ZSTACK_PBTOhNGN2hiPSCs_BR1_N19_Unmixed,lyso,8,7080,3002.281250,0,128,6.252741,1.656250,...,0,[],0,[],0,[],0,[],0,[]
4,4,ZSTACK_PBTOhNGN2hiPSCs_BR1_N19_Unmixed,lyso,9,3976,2270.150000,863,20,3.367781,1.000000,...,0,[],0,[],0,[],3,[1],0,[]


In [110]:
mito_table.volume.mean()

518.016